In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class WebDriverManager:
    driver_instance = None

    @classmethod
    def setup(cls):
        cls.driver_instance = webdriver.Chrome()
        cls.driver_instance.maximize_window()
        cls.driver_instance.implicitly_wait(10)
        cls.driver_instance.get("https://app-staging.nokodr.com/")

    @classmethod
    def teardown(cls):
        if cls.driver_instance:
            cls.driver_instance.quit()

    @classmethod
    def execute_signup(cls):
        wait = WebDriverWait(cls.driver_instance, 10)
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "(//input[contains(@class, 'slds-input')])[3]")))
        wait.until(EC.element_to_be_clickable(
            (By.CLASS_NAME, "slds-checkbox__label"))).click()
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@class='slds-col slds-size_1-of-1']"))).click()

class AuthenticationPage(WebDriverManager):

    @classmethod
    def login_user(cls, user_email, user_pass):
        cls.setup()
        email = user_email.strip()
        password = user_pass.strip()

        wait = WebDriverWait(cls.driver_instance, 10)
        email_field = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@name='username']")))
        password_field = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@name='password']")))
        
        email_field.send_keys(email)
        password_field.send_keys(password)
        
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@title='Log In']"))).click()

        try:
            error_box = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//div[@class='content-margin']")))
            message = error_box.text
            if "Invalid Email or Password" in message:
                print(f"❌ Login failed for {email}: Invalid credentials.")
            elif "Please enter a valid email" in message:
                print(f"❌ Login failed for {email}: Incorrect email format.")
            else:
                print(f"❌ Login encountered an unexpected error for {email}: {message}")
        except Exception:
            print(f"✅ Login successful for {email}!")
        finally:
            cls.teardown()

if __name__ == "__main__":
    
    test_credentials = [
        ("john.doe@gmail.com", "JohnDoe@123"),
        ("rajchautre@gmail.com", "Raj@7575"),
        ("user@example.com", "password123")
    ]

    for email, pwd in test_credentials:
        print(f"\n🔍 Testing login: Email: '{email}', Password: '{pwd}'")
        AuthenticationPage.login_user(email, pwd)



🔍 Testing login: Email: 'john.doe@gmail.com', Password: 'JohnDoe@123'
✅ Login successful for john.doe@gmail.com!

🔍 Testing login: Email: 'rajchautre@gmail.com', Password: 'Raj@7575'
✅ Login successful for rajchautre@gmail.com!

🔍 Testing login: Email: 'user@example.com', Password: 'password123'
❌ Login failed for user@example.com: Invalid credentials.
